In [15]:
import pandas as pd
import os
import datetime
from openpyxl import load_workbook
pd.options.mode.chained_assignment = None

current_date = datetime.datetime.now().strftime('%Y%m')  # 获取当前年月格式2023xx
now = now = datetime.datetime.now()
last_month = datetime.datetime(now.year, now.month-1, 1).strftime('%Y%m')
# monthfirstday = datetime.datetime.now().strftime('%Y%m%d')
# monthfirstday
today = datetime.datetime.today()
# monthfirstday = datetime.datetime(today.year, today.month, 1, 0, 0, 0).strftime('%Y%m%d %H:%M:%S')

path1 = r"C:\Users\hank\PythonCode\Monthlybill"
files = os.listdir(path1)
pattern = 'HW RTU_Monthly Report_'+ current_date
for f in files:
    if pattern in f:
        path = f
# 模糊查找，文件夹下含有'CHW RTU_Monthly Report_'+current_date的文件
filepath = path1+"\\"+path

# 需要读取上一月的数据
pattern_lastMonth = 'HW RTU_Monthly Report_'+last_month
for f in files:
    if pattern_lastMonth in f:
        path_last_month = f
filepath_lastmonth = path1+"\\"+path_last_month

df = pd.read_excel(filepath,
                   sheet_name='Report',
                   header=1,
                   parse_dates=['datetime']
                   )
df = df.dropna()  # 删除NaN
df_lastmonth = pd.read_excel(filepath_lastmonth,
                             sheet_name='Report',
                             header=1,
                             parse_dates=['datetime']
                             )
df_lastmonth = df_lastmonth.dropna()  # 删除NaN


In [16]:
df = pd.concat([df_lastmonth.tail(1), df[1:]])  # 用上月最后一行替换当月第一行
df.reset_index(drop=True, inplace=True)



In [17]:
df.columns

Index(['datetime', 'H1_QQ_9350_BTU', 'H2_QQ_9350_BTU', 'H3_QQ_9350_BTU',
       'H4_QQ_9350_BTU', 'H5_QQ_9350_BTU', 'H6_QQ_9350_BTU', 'H7_QQ_9350_BTU',
       'H8_QQ_9350_BTU', 'H9_QQ_9350_BTU',
       ...
       'PW_MH20_LD2', 'PW_MH20_LD2_Counter', 'HT_PH1_LD1',
       'HT_PH1_LD1_Counter', 'HT_PH1_LD2', 'HT_PH1_LD2_Counter',
       'HT_L_MH5_LD1', 'HT_L_MH5_LD1_Counter', 'HT_L_MH5_LD2',
       'HT_L_MH5_LD2_Counter'],
      dtype='object', length=157)

In [18]:

# 获取每日最大负荷QI
df_max_daily_QI = df.groupby(pd.Grouper(key='datetime', axis=0,
                                        freq='D')).max()  # 用于每天的最大负荷需求
df_max_daily_QI.drop(df_max_daily_QI.tail(1).index, inplace=True)  # 删除最后一行

df.drop(df.columns[110:158], axis=1, inplace=True)  # 删除后面的列
# df.drop(df.index[0], inplace=True)  # 删除第一行
daymin = df.groupby(pd.Grouper(key='datetime', axis=0,
                               freq='D')).min()
daymin_s = daymin[:-1]
daymin_e = daymin[1:]


data_0 = df[df["datetime"].dt.hour == 0]  # .dt获取每天某小时的数据
data_0_s = data_0[:-1]
data_0_e = data_0[1:]
data_0_e.reset_index(inplace=True, drop=True)
data_0_e.index = data_0_e.index + 1
data_0_s.reset_index(
    inplace=True, drop=True
)  # 为了方便用pandas的减法，需要将index重置，这里用到reset_index
# data_0_e=data_0_e.drop(columns=[Index])
# data_0_s=data_0_s.drop(columns=[Index])
data_0_s.index = data_0_s.index + 1
dayuse = (
    data_0_e
    - data_0_s
)
# dayuse.drop(dayuse['datetime'],axis=1,inplace=True)
# dayuse.index = daymin_s.index


In [19]:
df

,datetime,H1_QQ_9350_BTU,H2_QQ_9350_BTU,H3_QQ_9350_BTU,H4_QQ_9350_BTU,H5_QQ_9350_BTU,H6_QQ_9350_BTU,H7_QQ_9350_BTU,H8_QQ_9350_BTU,H9_QQ_9350_BTU,...,H11_TE_9310_2_BTU,H20_FT_9310,H20_FT_9310_BTU,H20_QI_9350_BTU,H20_QQ_9350_30min,H20_TE_9310_1,H20_TE_9310_1_BTU,H20_TE_9310_2,H20_TE_9310_2_BTU,HT_H_HH3_LD1
0,2023-06-01 00:00:00,106,511,14597148,1274013,1408034,6238246,559324,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,24.0,24.0,25.0,25.0,0.0
1,2023-06-01 01:00:00,106,511,14597469,1274038,1408034,6238343,559330,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,24.0,24.0,25.0,25.0,0.0
2,2023-06-01 02:00:00,106,511,14597794,1274061,1408034,6238453,559335,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,24.0,24.0,25.0,25.0,0.0
3,2023-06-01 03:00:00,106,511,14598084,1274080,1408034,6238561,559354,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,24.0,24.0,25.0,25.0,0.0
4,2023-06-01 04:00:00,106,511,14598470,1274101,1408034,6238671,559370,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,24.0,24.0,25.0,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,2023-06-30 20:00:00,106,511,14827152,1304790,1408161,6312605,569495,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,26.0,26.0,26.0,26.0,0.0
717,2023-06-30 21:00:00,106,511,14827386,1304860,1408161,6312702,569517,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,26.0,26.0,26.0,26.0,0.0
718,2023-06-30 22:00:00,106,511,14827641,1304916,1408161,6312806,569521,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,26.0,26.0,26.0,26.0,0.0
719,2023-06-30 23:00:00,106,511,14827975,1304959,1408161,6312934,569538,807618,3236951,...,34.0,0.0,0.0,0.0,0.0,26.0,26.0,26.0,26.0,0.0


In [20]:
dayuse.reset_index(drop=True, inplace=True)
dayuse.drop(columns=['datetime'], axis=1, inplace=True)
dayuse.set_index(daymin_s.index, inplace=True, verify_integrity=True)


In [21]:

wb2 = load_workbook("RCCQ HW Metering Report.xlsx")
# sheet_name = wb2.sheetnames
# print(sheet_name)


def write_dayuse_to_excel(shtname, dfname):
    alist = dfname
    ws = wb2[shtname]
    # cell_range = ws['E21':'E51']
    i = 0
    while i < len(alist):
        ws.cell(row=i + 21, column=5, value=alist[i])
        i += 1


def write_day_max_demand(shtname, dfname):
    alist = dfname
    ws = wb2[shtname]
    # cell_range = ws['C21':'C51']
    i = 0
    while i < len(alist):
        ws.cell(row=i + 21, column=3, value=alist[i])
        i += 1


def write_date(shtname):
    ws = wb2[shtname]
    ws['F6'] = datetime.datetime(now.year, now.month-1, 1).strftime('%b %Y')


def write_start_value(shtname, startvalue):
    ws = wb2[shtname]
    ws['E9'] = startvalue
    # ws['E8'] = endvalue


def write_end_value(shtname, endvalue):
    ws = wb2[shtname]
    ws['E8'] = endvalue

def write_start_date(shtname):
    ws = wb2[shtname]
    ws['B8'] = datetime.datetime(now.year, now.month-1, 1).strftime('%d.%m.%y')

def write_end_date(shtname):
    ws = wb2[shtname]
    ws['B9'] = datetime.datetime(now.year, now.month, 1).strftime('%d.%m.%y')

write_date('Summary')
write_end_date('HotelHZ HT')
write_start_date('HotelHZ HT')
# worksheet = wb2['PodiumE']
# A1 = worksheet['E9']
# print(A1.value)
write_start_value('HotelHZ HT', df_lastmonth.H3_QQ_9350_BTU.max())
write_start_value('HotelLZ HT', df_lastmonth.H4_QQ_9350_BTU.max())
write_start_value('HotelLZ MT', df_lastmonth.H5_QQ_9350_BTU.max())
write_start_value('SvcAptHZ HT', df_lastmonth.H6_QQ_9350_BTU.max())
write_start_value('SvcAptLZ HT', df_lastmonth.H7_QQ_9350_BTU.max())
write_start_value('SvcAptLZ MT', df_lastmonth.H8_QQ_9350_BTU.max())
write_start_value('OfficeHZ MT', df_lastmonth.H9_QQ_9350_BTU.max())
write_start_value('OfficeLZ MT', df_lastmonth.H10_QQ_9350_BTU.max())
write_start_value('PodiumE MT', df_lastmonth.H11_QQ_9350_BTU.max())
write_start_value('PodiumW MT', df_lastmonth.H20_QQ_9350_BTU.max())

write_end_value('HotelHZ HT', df.H3_QQ_9350_BTU.max())
write_end_value('HotelLZ HT',  df.H4_QQ_9350_BTU.max())
write_end_value('HotelLZ MT',  df.H5_QQ_9350_BTU.max())
write_end_value('SvcAptHZ HT', df.H6_QQ_9350_BTU.max())
write_end_value('SvcAptLZ HT', df.H7_QQ_9350_BTU.max())
write_end_value('SvcAptLZ MT',  df.H8_QQ_9350_BTU.max())
write_end_value('OfficeHZ MT',  df.H9_QQ_9350_BTU.max())
write_end_value('OfficeLZ MT',  df.H10_QQ_9350_BTU.max())
write_end_value('PodiumE MT',  df.H11_QQ_9350_BTU.max())
write_end_value('PodiumW MT',  df.H20_QQ_9350_BTU.max())


write_dayuse_to_excel('HotelHZ HT', dayuse['H3_QQ_9350_BTU'])
write_dayuse_to_excel('HotelLZ HT', dayuse['H4_QQ_9350_BTU'])
write_dayuse_to_excel('HotelLZ MT', dayuse['H5_QQ_9350_BTU'])
write_dayuse_to_excel('SvcAptHZ HT', dayuse['H6_QQ_9350_BTU'])
write_dayuse_to_excel('SvcAptLZ HT', dayuse['H7_QQ_9350_BTU'])
write_dayuse_to_excel('SvcAptLZ MT', dayuse['H8_QQ_9350_BTU'])
write_dayuse_to_excel('OfficeHZ MT', dayuse['H9_QQ_9350_BTU'])
write_dayuse_to_excel('OfficeLZ MT', dayuse['H10_QQ_9350_BTU'])
write_dayuse_to_excel('PodiumE MT', dayuse['H11_QQ_9350_BTU'])
write_dayuse_to_excel('PodiumW MT', dayuse['H20_QQ_9350_BTU'])

write_day_max_demand('HotelHZ HT', df_max_daily_QI['H3_QI_9350_BTU'])
write_day_max_demand('HotelLZ HT', df_max_daily_QI['H4_QI_9350_BTU'])
write_day_max_demand('HotelLZ MT', df_max_daily_QI['H5_QI_9350_BTU'])
write_day_max_demand('SvcAptHZ HT', df_max_daily_QI['H6_QI_9350_BTU'])
write_day_max_demand('SvcAptLZ HT', df_max_daily_QI['H7_QI_9350_BTU'])
write_day_max_demand('SvcAptLZ MT', df_max_daily_QI['H8_QI_9350_BTU'])
write_day_max_demand('OfficeHZ MT', df_max_daily_QI['H9_QI_9350_BTU'])
write_day_max_demand('OfficeLZ MT', df_max_daily_QI['H10_QI_9350_BTU'])
write_day_max_demand('PodiumE MT', df_max_daily_QI['H11_QI_9350_BTU'])
write_day_max_demand('PodiumW MT', df_max_daily_QI['H20_QI_9350_BTU'])
wb2.save("RCCQ HW Metering Report.xlsx")
